# SQL Options in Spark-PySpark

PySpark provides two main options when it comes to using straight SQL. Spark SQL and SQL Transformer

## SQL Options within regular PySpark Calls

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark

In [2]:
crime = spark.read.csv("rec-crime-pfa.csv", inferSchema=True, header=True)
crime.limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling year total number of offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [3]:
crime.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)



In [4]:
df = crime.withColumnRenamed("Rolling year total number of offences", "Count")
df.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Count: integer (nullable = true)



In [5]:
df.createOrReplaceTempView("tempView")

In [6]:
spark.sql("SELECT * FROM tempView").limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Count
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [7]:
spark.sql("SELECT Region, SUM(Count) AS Total FROM tempView GROUP BY Region ORDER BY Total DESC").toPandas()

,Region,Total
0,London,42691902
1,South East,30911995
2,North West,30235732
3,Yorkshire and The Humber,23769180
4,West Midlands,21040955
5,East,19890612
6,South West,17985880
7,East Midlands,17264827
8,Wales,11137260
9,North East,10236642


## SQL Transformer

In [8]:
from pyspark.ml.feature import SQLTransformer

In [9]:
sql_transformation = SQLTransformer(statement="SELECT PFA, Region, Offence FROM __THIS__")
sql_transformation.transform(df).show(5)

+-----------------+----------+--------------------+
|              PFA|    Region|             Offence|
+-----------------+----------+--------------------+
|Avon and Somerset|South West|All other theft o...|
|Avon and Somerset|South West|       Bicycle theft|
|Avon and Somerset|South West|Criminal damage a...|
|Avon and Somerset|South West|Death or serious ...|
|Avon and Somerset|South West|   Domestic burglary|
+-----------------+----------+--------------------+
only showing top 5 rows



In [10]:
type(sql_transformation)

pyspark.ml.feature.SQLTransformer

In [11]:
sql_transformation = SQLTransformer(statement="SELECT Offence, SUM(Count) AS Total FROM __THIS__ GROUP BY Offence")
sql_transformation.transform(df).toPandas()

,Offence,Total
0,Public order offences,10925676
1,Bicycle theft,5297006
2,Residential burglary,1671469
3,Violence without injury,16590158
4,All other theft offences,30979393
5,Robbery,3788128
6,CIFAS,7678981
7,Fraud offences,2596554
8,Sexual offences,4006741
9,Criminal damage and arson,37767463


In [12]:
result = sql_transformation.transform(df)
result.show()

+--------------------+--------+
|             Offence|   Total|
+--------------------+--------+
|Public order offe...|10925676|
|       Bicycle theft| 5297006|
|Residential burglary| 1671469|
|Violence without ...|16590158|
|All other theft o...|30979393|
|             Robbery| 3788128|
|               CIFAS| 7678981|
|      Fraud offences| 2596554|
|     Sexual offences| 4006741|
|Criminal damage a...|37767463|
|            Homicide|   34154|
|Possession of wea...| 1555951|
|          UK Finance| 2925861|
|Stalking and hara...| 5587434|
|Theft from the pe...| 5105153|
|         Shoplifting|16781641|
|       Drug offences| 9999435|
|    Vehicle offences|26075670|
|   Domestic burglary|11694636|
|Miscellaneous cri...| 3143136|
+--------------------+--------+
only showing top 20 rows



## Column Functions

In [13]:
total_data = SQLTransformer(statement="SELECT SUM(Count) AS Total FROM __THIS__")
total = total_data.transform(df).collect()[0]["Total"]


In [14]:
from pyspark.sql.functions import expr

In [15]:
df.withColumn("percent", expr(f"round((count/{total}) * 100, 2)")).show()

+----------------+-----------------+----------+--------------------+-----+-------+
|12 months ending|              PFA|    Region|             Offence|Count|percent|
+----------------+-----------------+----------+--------------------+-----+-------+
|      31/03/2003|Avon and Somerset|South West|All other theft o...|25959|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Bicycle theft| 3090|    0.0|
|      31/03/2003|Avon and Somerset|South West|Criminal damage a...|26202|   0.01|
|      31/03/2003|Avon and Somerset|South West|Death or serious ...|    2|    0.0|
|      31/03/2003|Avon and Somerset|South West|   Domestic burglary|14561|   0.01|
|      31/03/2003|Avon and Somerset|South West|       Drug offences| 2308|    0.0|
|      31/03/2003|Avon and Somerset|South West|      Fraud offences| 5339|    0.0|
|      31/03/2003|Avon and Somerset|South West|            Homicide|   19|    0.0|
|      31/03/2003|Avon and Somerset|South West|Miscellaneous cri...| 1597|    0.0|
|   

In [16]:
df.select("*", expr(f"round((count/{total}) * 100, 2)")).toPandas()

,12 months ending,PFA,Region,Offence,Count,"round(((CAST(count AS DOUBLE) / CAST(244720928 AS DOUBLE)) * CAST(100 AS DOUBLE)), 2)"
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.01
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.00
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.01
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.00
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561,0.01
...,...,...,...,...,...,...
46464,31/12/2018,Wiltshire,South West,Stalking and harassment,2380,0.00
46465,31/12/2018,Wiltshire,South West,Theft from the person,347,0.00
46466,31/12/2018,Wiltshire,South West,Vehicle offences,2895,0.00
46467,31/12/2018,Wiltshire,South West,Violence with injury,5701,0.00


In [17]:
df.selectExpr("*", f"round((count/{total}) * 100, 2) AS Percent").filter("Region = 'South West'").toPandas()

,12 months ending,PFA,Region,Offence,Count,Percent
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.01
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.00
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.01
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.00
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561,0.01
...,...,...,...,...,...,...
5265,31/12/2018,Wiltshire,South West,Stalking and harassment,2380,0.00
5266,31/12/2018,Wiltshire,South West,Theft from the person,347,0.00
5267,31/12/2018,Wiltshire,South West,Vehicle offences,2895,0.00
5268,31/12/2018,Wiltshire,South West,Violence with injury,5701,0.00
